In [ ]:
import h5py
import numpy as np
from numpy import zeros, ones, asarray, concatenate, array, asarray, pad
import os, sys
from PIL import Image, ImageOps
import cv2
from h5py import File
from math import ceil, floor
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# BASE_DIR = os.pardir
file = os.path.join('/content','drive','MyDrive','BHSig260','BHSig260','Hindi','list.genuine')
print(file)
with open(file) as fil:
    genuine_raw = list(fil.read().split('\n'))
genuine = list()
for lin in genuine_raw:
    genuine.append(lin[lin.find('/') + 1:])#appeding name of the image

file = os.path.join('/content','drive','MyDrive','BHSig260','BHSig260','Hindi', 'list.forgery')
with open(file) as fil:
    forged_raw = list(fil.read().split('\n'))
forged = list()
for lin in forged_raw:
    forged.append(lin[lin.find('/') + 1:])

/content/drive/MyDrive/BHSig260/BHSig260/Hindi/list.genuine


In [ ]:
# %%timeit -n1 -r1
# BASE_DIR = os.pardir
i = 0
x, y = 224,224
X = np.zeros((54 * 160, y, x, 3), dtype=np.int8)
Y = np.zeros((54 * 160, 1), dtype = np.uint8)
S = np.zeros((54 * 160, 1), dtype = np.uint8)

for indir in os.listdir(os.path.join('/content','drive','MyDrive','BHSig260','BHSig260','Bengali')):
    PATH = os.path.join('/content','drive','MyDrive','BHSig260','BHSig260','Bengali', indir)
    try:
        for infile in os.listdir(PATH):
            im = cv2.imread((os.path.join(PATH,infile)),0)
#             with cv.imread((os.path.join(PATH, infile)),0) as im:
            X[i] = (cv2.resize(im,(x,y), interpolation = cv2.INTER_AREA)).reshape(y,x,1)
            S[i] = int(indir)
            if infile in genuine:
                Y[i] = 1
            # elif infile in forged:
                # Y[i] = 0
            i = i + 1
        print(i/54, end = '\t')
    except:
        print("{} might not be a valid directory".format(indir))

1.0	2.0	3.0	4.0	5.0	6.0	7.0	8.0	9.0	10.0	086 might not be a valid directory
11.203703703703704	12.203703703703704	13.203703703703704	14.203703703703704	15.203703703703704	

In [ ]:
# im = cv2.imread(os.path.join(PATH,infile),0)
# im.shape

In [ ]:
file = '/content/drive/MyDrive/bhsig260bengali_224x224.h5'
with h5py.File(file, 'w') as hdf:
    hdf.create_dataset('X', data=X)
    hdf.create_dataset('Y', data=Y)
    hdf.create_dataset('S', data=S)

In [ ]:
# print(sum(Y[0:54]))

In [ ]:
# x, y = 224,224
def xor(x, y):
    z = x ^ y
    return z

In [ ]:
# BASE_DIR = os.getcwd()
file = '/content/drive/MyDrive/bhsig260hindi_224x224x3.h5'
# file = os.path.join(BASE_DIR, infile)
with File(file, 'r') as hdf:
    X = array(hdf.get('X'))
    Y = array(hdf.get('Y'))
    S = array(hdf.get('S'))
writers = list()
for i in S.squeeze():
    if i not in writers:
        writers.append(i)
print(writers)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160]


In [ ]:
X.shape

(8640, 224, 224, 3)

In [ ]:
# Taking only data of 30 writers bcoz of memory limitations
number_of_writers = 15
writers = writers[0:number_of_writers]
X = X[0:number_of_writers*54]
Y = Y[0:number_of_writers*54]
S = S[0:number_of_writers*54]

print(len(writers),len(X),len(Y),len(S))

15 810 810 810


In [ ]:
# n_original * n_forged

In [ ]:
n_writers = len(writers)
print(n_writers)
n_original = int(Y.sum() // n_writers)
print(n_original)
n_forged = int(len(Y.squeeze()) // n_writers - n_original)
print(n_forged)
m = int(n_writers * (n_original * n_forged + n_original * (n_original - 1) // 2))
print(m)

15
24
30
14940


In [ ]:
S1 = np.zeros((m, X.shape[1], X.shape[2], X.shape[3]), dtype = np.uint8)
S2 = np.zeros((m, X.shape[1], X.shape[2], X.shape[3]), dtype = np.uint8)
y = np.zeros((m, 1), dtype = np.uint8)
w = np.zeros((m, 1), dtype = np.uint8)
l = 0
for writer in writers:
    ind = np.where(S.squeeze() == writer)
    if len(ind) != 1:
        print('Error')
    for i in ind[0]:
        for j in ind[0]:
            if i <= j:
                continue
            if Y.squeeze()[i] == 0 and Y.squeeze()[j] == 0:
                continue
            S1[l] = X[i]
            S2[l] = X[j]
            y[l] = xor(Y[i], Y[j])
            w[l][0] = writer
            l = l + 1
            # 'bhsig260hindi_224x224_Preprocessed_drive.h5'
outfile = '/content/drive/MyDrive/'+'bigsig260'+str(X.shape[1])+'x'+str(X.shape[2])+'x'+str(X.shape[3])+'_siamese_preprocessed.h5'
print(outfile)
# file = os.path.join(BASE_DIR, outfile)
with File(outfile, 'w') as hdf:
    hdf.create_dataset('S1', data = S1)
    hdf.create_dataset('S2', data = S2)
    hdf.create_dataset('Y',   data = y)
    hdf.create_dataset('L',   data = w)


/content/drive/MyDrive/bigsig260224x224x3_siamese_preprocessed.h5
